In [2]:
import requests
from tqdm import tqdm
import csv
import json

In [3]:
url = "https://lks.bmstu.ru/lks-back/api/v1/structure"

response = requests.get(url)
response = response.json()

In [4]:
mainMGTU = response['data']['children'][0]
kalugaMGTU = response['data']['children'][1]
mytishchiMGTU = response['data']['children'][2]

In [ ]:
groups = []
for faculty in mainMGTU['children']:
  for department in faculty['children']:
    if (department['children']):
      for course in department['children']:
        for group in course['children']:
          groups.append([group['abbr'], group['uuid']])

In [7]:
schedules = []

scheduleUrl = 'https://lks.bmstu.ru/lks-back/api/v1/schedules/groups/'

for group in tqdm(groups):
  req = requests.get(scheduleUrl + group[1] + '/public')
  if (req.ok and req.status_code == 200):
    req = req.json()
    if (req['data']['schedule']):
      schedules.append([req['data']['title'], req['data']['schedule']])
  


100%|██████████| 3285/3285 [03:26<00:00, 15.90it/s]


In [ ]:
schedule = {}
for group in schedules:
  for lesson in group[1]:
    if (lesson['teachers']):
      if group[0] not in schedule:
        schedule[group[0]] = []
      schedule[group[0]].append((lesson['discipline']['fullName'], f"{lesson['teachers'][0]['lastName']} {lesson['teachers'][0]['firstName']} {lesson['teachers'][0]['middleName'] if lesson['teachers'][0]['middleName'] is not None else ''}".strip()))

with open("gr_teach_subj.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Group","Teacher","Subject"])
    
    for group, pairs in schedule.items():
        print(f"Группа: {group}")
        for subject, teacher in pairs:
            print(f"  Предмет: {subject}, Преподаватель: {teacher}")
            writer.writerow([group, teacher, subject]) 

Группа: АК1-41
  Предмет: Метрология, стандартизация и сертификация, Преподаватель: Иванина Ирина Владимировна
  Предмет: Физика, Преподаватель: Селиванов Алексей Борисович
  Предмет: Метрология, стандартизация и сертификация, Преподаватель: Иванина Ирина Владимировна
  Предмет: Физика, Преподаватель: Селиванов Алексей Борисович
  Предмет: Теория механизмов и машин, Преподаватель: Ибаева Любовь Викторовна
  Предмет: Метрология, стандартизация и сертификация, Преподаватель: Минязева Луиза Халисовна
Группа: АК1-42
  Предмет: Метрология, стандартизация и сертификация, Преподаватель: Минязева Луиза Халисовна
Группа: АК1-61
  Предмет: Двигательные установки, Преподаватель: Елисеев Алексей Алексеевич
Группа: АК1-62
  Предмет: Двигательные установки, Преподаватель: Елисеев Алексей Алексеевич
Группа: АК1-81
  Предмет: Автоматизированная компоновка пилотируемых и автоматических космических аппаратов и систем, Преподаватель: Шаповалов Анатолий Витальевич
  Предмет: Автоматизированная компоновка 